In [4]:
import numpy as np
from scipy.optimize import minimize_scalar

# Целевая функция f(x)
def f(x):
    # Пример квадратичной функции, замените на нужную
    return x[0]**2 + x[1]**2

# Градиент функции f(x)
def grad_f(x):
    # Пример для функции f(x) = x[0]2 + x[1]2
    return np.array([2 * x[0], 2 * x[1]])

# Метод Флетчера-Ривза и Полака-Рибьера
def conjugate_gradient(x0, epsilon1=1e-5, delta2=1e-5, epsilon2=1e-5, M=1000, method="FR"):
    x_k = np.array(x0)
    grad_k = grad_f(x_k)
    d_k = -grad_k  # Начальное направление
    k = 0

    while k < M:
        # Проверка критерия окончания по градиенту
        if np.linalg.norm(grad_k) < epsilon1:
            print(f"Оптимум достигнут на {k}-й итерации")
            return x_k
        
        # Определение шага альфа через одномерную минимизацию
        def phi(alpha):
            return f(x_k + alpha * d_k)
        
        res = minimize_scalar(phi, bounds=(0, 1), method='bounded')
        alpha_k = res.x

        # Обновление x
        x_next = x_k + alpha_k * d_k
        grad_next = grad_f(x_next)

        # Выбор формулы для вычисления w^(k-1)
        if method == "FR":
            # Флетчер-Ривз
            w_k = np.linalg.norm(grad_next)**2 / np.linalg.norm(grad_k)**2
        elif method == "PR":
            # Полак-Рибьер
            w_k = max(
                np.dot(grad_next, grad_next - grad_k) / np.linalg.norm(grad_k)**2,
                0
            ) if k % len(x0) != 0 else 0

        # Вычисление нового направления d_k
        d_k = -grad_next + w_k * d_k

        # Проверка условий окончания по x и значению функции
        if np.linalg.norm(x_next - x_k) < delta2 and abs(f(x_next) - f(x_k)) < epsilon2:
            print(f"Условия окончания достигнуты на {k}-й итерации")
            return x_next

        # Подготовка к следующей итерации
        x_k = x_next
        grad_k = grad_next
        k += 1
    
    print("Достигнуто предельное число итераций")
    return x_k

# Начальная точка и параметры алгоритма
x0 = [10, 10]
result_fr = conjugate_gradient(x0, method="FR")
print("Результат (Флетчер-Ривз):", result_fr)

result_pr = conjugate_gradient(x0, method="PR")
print("Результат (Полак-Рибьер):", result_pr)

Оптимум достигнут на 1-й итерации
Результат (Флетчер-Ривз): [0. 0.]
Оптимум достигнут на 1-й итерации
Результат (Полак-Рибьер): [0. 0.]
